In [1]:
import boto3
import os
import dotenv

dotenv.load_dotenv()

True

In [9]:
def check_aws_credentials():
    """
    Check if the required AWS credentials are set as environment variables.
    """
    required_vars = ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY']
    missing_vars = [var for var in required_vars if not os.getenv(var)]

    if missing_vars:
        raise ValueError("AWS credentials are not set")

check_aws_credentials()

In [4]:
import yaml

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

print(config)

{'kinesis': {'source_stream_name': 'nullify-source-stream', 'enriched_stream_name': 'nullify-enriched-stream'}, 'sns': {'topic_name': 'nullify-threat-intel-topic'}}


In [2]:
def create_kinesis_stream(stream_name, shard_count=1):
    """
    Create a Kinesis stream in provisioned mode with the specified number of shards.

    :param stream_name: Name of the Kinesis stream to create
    :param shard_count: Number of shards for the stream (default is 1)
    :return: True if stream created successfully, False otherwise
    """
    try:
        kinesis_client = boto3.client('kinesis', region_name='ap-southeast-2')
        kinesis_client.create_stream(
            StreamName=stream_name,
            ShardCount=shard_count,
            StreamModeDetails={
                'StreamMode': 'PROVISIONED'
            }
        )
        print(f"Kinesis stream '{stream_name}' created successfully with {shard_count} shard(s).")
        return True
    except Exception as e:
        print(f"An error occurred while creating the Kinesis stream: {e}")
        return False

In [5]:
create_kinesis_stream(config['kinesis']['source_stream_name'])
create_kinesis_stream(config['kinesis']['enriched_stream_name'])

Kinesis stream 'nullify-source-stream' created successfully with 1 shard(s).
Kinesis stream 'nullify-enriched-stream' created successfully with 1 shard(s).


True

In [ ]:
## Create SNS topic
sns_client = boto3.client('sns', region_name='ap-southeast-2')

sns_client.create_topic(Name=config['sns']['topic_name'])


In [6]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from requests_aws4auth import AWS4Auth
import boto3

service = 'aoss'
region = 'ap-southeast-2'
service = 'es'
username = 'nullify'
password = os.getenv('OPENSEARCH_PASSWORD')

host = 'search-nullify-test-senb2mnquk4zcaqdlnubnv5g2y.aos.ap-southeast-2.on.aws'

# create an opensearch client and use the username and password for authentication
client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=(username, password),
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    pool_maxsize=20,
)

# create an index
index_name = 'books-index'
create_response = client.indices.create(
    index_name,
    body={
        'settings': {
            'number_of_shards': 1,
            'number_of_replicas': 0
        },
        'mappings': {
            'properties': {
                'id': {'type': 'keyword'},
                'title': {'type': 'text'},
                'director': {'type': 'text'},
                'year': {'type': 'text'}
            }
        },
    }
)

print('\nCreating index:')
print(create_response)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'books-index'}


In [8]:
# index a document
documents = [
  {
      'id': '1',
  'title': 'The Green Mile',
    'director': 'Stephen King',
    'year': '1996'
  },
  {
    'id': '2',
    'title': 'The Shawshank Redemption',
    'director': 'Stephen King',
    'year': '1994'
}, {
    'id': '3',
    'title': 'The Godfather',
    'director': 'Francis Ford Coppola',
    'year': '1972'
}]

for d in documents:
  response = client.index(
      index = 'books-index',
      body = d,
      id = d['id']
  )
  print(response)

{'_index': 'books-index', '_id': '1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
{'_index': 'books-index', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
{'_index': 'books-index', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
